<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=728770b039442be4245b9da5dd163f18aee7df7fdd30754d94c1a9cf0cde6463
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-21 15:19:34
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.41 C
-------------------
Today PnL: 45.84 K (0.33%)
Current PnL: -20.19 L (-13.45%)
CY Booked + Current PnL: -8.09 L (-5.39%)
-------------------
Total profit:  1.98 L
Total loss:  -22.16 L
-------------------
Total Booked + Current PnL: 18.89 L (15.35%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.55%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.39 L (61.77%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 8.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VOLTAS,1530.00,4.10,66.0,X-MC,2.02,218370.0,26628.0,11137.0,0.76,13.89,5.10,19.69,99.0,2.39,1.55,21.63,XY25,NTT,AC
58,RELIANCE,1533.00,-8.78,71.0,X-LC,2.85,227916.0,17550.0,11236.0,-0.40,8.34,4.93,13.68,37.0,1.56,1.62,26.24,XY25,NTT,REFINERIES
2,ABBOTINDIA,35195.00,-8.03,50.0,X-MC,3.64,90900.0,222.0,14689.0,1.24,0.24,16.16,16.44,101.0,0.02,0.65,20.47,X40,ATH,PHARMA
78,TTKPRESTIG,770.00,95.52,48.0,M-SC,1.84,84279.0,-16498.0,16595.0,0.93,-16.37,19.69,0.09,245.0,-0.99,0.60,10.72,OX40N,NTT,DURABLES
34,ICICIGI,2260.25,-13.61,64.0,X-MC,2.93,148518.0,12385.0,18743.0,-0.20,9.10,12.62,22.86,91.0,0.66,1.05,22.86,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BSOFT,836.99,-20.22,45.0,H-SC,15.44,89722.0,-50931.0,123709.0,3.81,-36.21,137.88,51.74,131.0,-0.41,0.64,3.81,XR,ATH,IT
83,VALIANTORG,838.00,-287.15,39.0,H-SC,6.58,104040.0,-59565.0,180884.0,3.08,-36.41,173.86,74.15,139.0,-0.33,0.74,33.36,XR,NTT,CHEMICALS
38,INDIGOPNTS,1408.00,98.10,43.0,M-SC,4.33,130051.0,-44508.0,44542.0,2.81,-25.50,34.25,0.02,221.0,-1.00,0.92,13.12,OX40N,NTT,PAINTS
64,SFL,1287.00,33.01,51.0,M-SC,7.36,175621.0,-86597.0,156426.0,1.95,-33.02,89.07,26.63,229.0,-0.55,1.25,9.19,XY24,NTT,MISC
37,INDIAMART,4810.62,-54.83,45.0,H-SC,7.54,126087.0,2751.0,128874.0,1.90,2.23,102.21,106.72,119.0,0.02,0.89,26.13,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.00,113.54,62.0,H-SC,13.60,135000.0,-7479.0,27000.0,-2.46,-5.25,20.00,13.70,163.0,-0.28,0.96,59.34,OX40N,NTT,BANKS
16,CAMPUS,393.00,-23.39,54.0,M-SC,2.88,159548.0,-8227.0,64074.0,-1.35,-4.90,40.16,33.28,210.0,-0.13,1.13,23.66,XY24,NTT,FOOTWEAR
48,LTIM,7230.20,0.58,62.0,H-LC,4.04,249548.0,3397.0,75813.0,-0.91,1.38,30.38,32.18,16.0,0.04,1.77,37.92,X200,ATH,IT
73,TATAELXSI,9161.00,-24.89,43.0,H-MC,7.62,101707.0,-30128.0,72354.0,-0.82,-22.85,71.14,32.03,98.0,-0.42,0.72,13.92,OX40N,NTT,IT
32,HINDZINC,730.22,29.16,51.0,M-LC,10.37,210083.0,5007.0,107562.0,-0.71,2.44,51.20,54.89,52.0,0.05,1.49,27.13,X5K,ATH,METALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.0,41.14,47.0,M-MC,10.59,223343.0,-1619.0,169272.0,-0.11,-0.72,75.79,74.53,192.0,-0.01,1.58,31.30,XY24,BTT,STEEL
87,WIPRO,420.0,-14.70,43.0,M-LC,6.17,149538.0,-1407.0,110867.0,-0.02,-0.93,74.14,72.51,53.0,-0.01,1.06,5.03,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,113.54,62.0,H-SC,13.60,135000.0,-7479.0,27000.0,-2.46,-5.25,20.00,13.70,163.0,-0.28,0.96,59.34,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.51,65.0,H-SC,2.00,230580.0,-39087.0,75423.0,1.05,-14.49,32.71,13.47,138.0,-0.52,1.64,17.24,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,95.52,48.0,M-SC,1.84,84279.0,-16498.0,16595.0,0.93,-16.37,19.69,0.09,245.0,-0.99,0.60,10.72,OX40N,NTT,DURABLES
18,CERA,9475.0,-23.39,44.0,H-SC,2.60,139610.0,-36293.0,78321.0,0.46,-20.63,56.10,23.89,149.0,-0.46,0.99,20.57,OX40N,NTT,CERAMICS
67,SIS,528.0,2075.65,53.0,H-SC,3.59,87809.0,-23223.0,46829.0,0.25,-20.92,53.33,21.26,156.0,-0.50,0.62,18.64,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,47.87,52.0,M-LC,6.08,207102.0,3932.0,38190.0,0.49,1.94,18.44,20.74,55.0,0.10,1.47,7.59,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.16,51.0,M-LC,10.37,210083.0,5007.0,107562.0,-0.71,2.44,51.20,54.89,52.0,0.05,1.49,27.13,X5K,ATH,METALS
46,KPIGREEN,731.05,6.72,67.0,H-SC,5.54,127197.0,1900.0,57022.0,1.64,1.52,44.83,47.03,141.0,0.03,0.90,59.26,MH,ATH,POWER
80,UNIONBANK,163.00,-11.63,58.0,M-LC,8.87,160658.0,19818.0,24677.0,-0.02,14.07,15.36,31.59,66.0,0.80,1.14,44.29,XY24,NTT,BANKS
12,BANKINDIA,190.00,-22.98,66.0,H-MC,13.02,200094.0,20286.0,92123.0,0.25,11.28,46.04,62.52,88.0,0.22,1.42,49.21,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-22.98,66.0,H-MC,13.02,200094.0,20286.0,92123.0,0.25,11.28,46.04,62.52,88.0,0.22,1.42,49.21,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-39.40,65.0,M-SC,4.12,102525.0,11026.0,67728.0,0.67,12.05,66.06,86.07,223.0,0.16,0.73,48.88,XR,NTT,DURABLES
32,HINDZINC,730.22,29.16,51.0,M-LC,10.37,210083.0,5007.0,107562.0,-0.71,2.44,51.20,54.89,52.0,0.05,1.49,27.13,X5K,ATH,METALS
46,KPIGREEN,731.05,6.72,67.0,H-SC,5.54,127197.0,1900.0,57022.0,1.64,1.52,44.83,47.03,141.0,0.03,0.90,59.26,MH,ATH,POWER
37,INDIAMART,4810.62,-54.83,45.0,H-SC,7.54,126087.0,2751.0,128874.0,1.90,2.23,102.21,106.72,119.0,0.02,0.89,26.13,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.83,15.0,X-LC,11.49,145509.0,-131014.0,241094.0,0.28,-47.38,165.69,39.81,54.0,-0.54,1.03,2.56,XY24,NTT,AUTO
24,DMART,5391.45,-16.17,37.0,X-LC,10.20,102528.0,-58.0,26862.0,-0.08,-0.06,26.20,26.13,38.0,-0.00,0.73,25.50,X40N,ATH,FMCG
53,QUESS,986.00,-47.66,37.0,X-SC,38.76,52122.0,-12884.0,163814.0,0.11,-19.82,314.29,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
29,HAVELLS,2069.16,-6.60,43.0,X-MC,2.62,231660.0,-16071.0,91135.0,-0.03,-6.49,39.34,30.30,92.0,-0.18,1.64,5.94,X40,ATH,ELECTRICAL
66,SIEMENS,4671.50,-3.47,43.0,H-LC,0.47,155255.0,-30840.0,78326.0,0.27,-16.57,50.45,25.51,15.0,-0.39,1.10,14.41,AR,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-3.47,43.0,H-LC,0.47,155255.0,-30840.0,78326.0,0.27,-16.57,50.45,25.51,15.0,-0.39,1.10,14.41,AR,ATH,ELECTRICAL
85,VOLTAS,1530.00,4.10,66.0,X-MC,2.02,218370.0,26628.0,11137.0,0.76,13.89,5.10,19.69,99.0,2.39,1.55,21.63,XY25,NTT,AC
42,ITC,452.00,-37.50,59.0,X-LC,2.54,201712.0,1574.0,19324.0,-0.13,0.79,9.58,10.44,4.0,0.08,1.43,6.39,X40,NTT,FMCG
29,HAVELLS,2069.16,-6.60,43.0,X-MC,2.62,231660.0,-16071.0,91135.0,-0.03,-6.49,39.34,30.30,92.0,-0.18,1.64,5.94,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-8.78,71.0,X-LC,2.85,227916.0,17550.0,11236.0,-0.40,8.34,4.93,13.68,37.0,1.56,1.62,26.24,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,37.0,X-SC,38.76,52122.0,-12884.0,163814.0,0.11,-19.82,314.29,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
74,TATAMOTORS,1065.00,-53.83,15.0,X-LC,11.49,145509.0,-131014.0,241094.0,0.28,-47.38,165.69,39.81,54.0,-0.54,1.03,2.56,XY24,NTT,AUTO
3,ACC,3906.00,-37.64,50.0,X-MC,3.14,185000.0,-52951.0,205609.0,1.04,-22.25,111.14,64.15,174.0,-0.26,1.31,3.78,XY24,BTT,CEMENT
75,TCS,4389.97,-29.33,50.0,X-LC,12.69,282000.0,-63956.0,130651.0,-0.50,-18.49,46.33,19.28,1.0,-0.49,2.00,4.25,X40,ATH,IT
10,BAJAJHFL,181.50,-13.84,49.0,X-MC,5.58,181597.0,-18909.0,116059.0,1.54,-9.43,63.91,48.45,90.0,-0.16,1.29,4.72,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,37.0,X-SC,38.76,52122.0,-12884.0,163814.0,0.11,-19.82,314.29,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
51,PAGEIND,51769.93,-26.33,44.0,X-MC,6.62,83230.0,-280.0,20308.0,-0.25,-0.34,24.40,23.98,82.0,-0.01,0.59,5.46,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.20,46.0,X-SC,4.84,82936.0,-62224.0,138155.0,1.58,-42.87,166.58,52.31,136.0,-0.45,0.59,11.01,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-8.03,50.0,X-MC,3.64,90900.0,222.0,14689.0,1.24,0.24,16.16,16.44,101.0,0.02,0.65,20.47,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.22,96944.0,-31726.0,77022.0,0.27,-24.66,79.45,35.20,219.0,-0.41,0.69,11.27,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.33,50.0,X-LC,12.69,282000.0,-63956.0,130651.0,-0.50,-18.49,46.33,19.28,1.0,-0.49,2.00,4.25,X40,ATH,IT
40,INFY,2275.00,-19.60,48.0,X-LC,7.60,313323.0,435.0,171262.0,0.69,0.14,54.66,54.87,3.0,0.00,2.22,6.80,X40,BTT,IT
42,ITC,452.00,-37.50,59.0,X-LC,2.54,201712.0,1574.0,19324.0,-0.13,0.79,9.58,10.44,4.0,0.08,1.43,6.39,X40,NTT,FMCG
84,VBL,671.64,-18.69,52.0,X-LC,10.39,293033.0,-22809.0,135469.0,0.65,-7.22,46.23,35.67,5.0,-0.17,2.08,5.63,X40N,ATH,FMCG
1,ABB,7934.00,-39.87,53.0,H-LC,6.70,251136.0,-10483.0,129687.0,0.01,-4.01,51.64,45.57,7.0,-0.08,1.78,6.86,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7150.00,69.0,L-SC,6.72,79566.0,-14244.0,90864.0,1.12,-15.18,114.20,81.67,269.0,-0.16,0.56,54.08,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,52.0,L-SC,30.10,82560.0,-30989.0,71043.0,0.94,-27.29,86.05,35.27,268.0,-0.44,0.59,102.07,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,52.0,H-SC,9.48,92400.0,-5580.0,27184.0,1.48,-5.70,29.42,22.05,152.0,-0.21,0.66,34.31,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-39.40,65.0,M-SC,4.12,102525.0,11026.0,67728.0,0.67,12.05,66.06,86.07,223.0,0.16,0.73,48.88,XR,NTT,DURABLES
46,KPIGREEN,731.05,6.72,67.0,H-SC,5.54,127197.0,1900.0,57022.0,1.64,1.52,44.83,47.03,141.0,0.03,0.90,59.26,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.0,-190.91,75.0,M-SC,21.07,157095.0,7875.0,136311.0,1.49,5.28,86.77,96.62,208.0,0.06,1.11,70.01,XY25,NTT,FINANCE
70,SURYODAY,240.0,66.67,63.0,H-SC,11.88,153765.0,-25306.0,91029.0,1.26,-14.13,59.20,36.71,135.0,-0.28,1.09,52.57,XR,NTT,BANKS
79,UJJIVANSFB,60.0,113.54,62.0,H-SC,13.60,135000.0,-7479.0,27000.0,-2.46,-5.25,20.00,13.70,163.0,-0.28,0.96,59.34,OX40N,NTT,BANKS
59,REPCOHOME,880.0,-56.04,69.0,H-SC,2.22,253998.0,-32507.0,289837.0,1.06,-11.35,114.11,89.82,134.0,-0.11,1.80,34.38,XY24,NTT,FINANCE
5,ANGELONE,3033.0,15.31,68.0,X-SC,7.71,204016.0,13010.0,44700.0,-0.46,6.81,21.91,30.21,157.0,0.29,1.45,29.11,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.42
1,25,43.83
2,50,74.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.44
LC    32.70
MC    22.87
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.42
X40      15.93
XY25     12.35
XR       11.11
X40N      9.34
AR        7.96
OX40N     7.63
X200      1.77
X5K       1.49
SR        1.11
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.53
X-LC    21.86
X-MC    17.18
M-SC    12.78
M-LC     5.16
H-LC     4.65
X-SC     4.64
H-MC     3.80
M-MC     1.58
L-SC     1.49
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.80,-3.36,37.09
IT,12.56,-18.84,82.98
FINANCE,12.21,-9.90,59.72
MISC,6.94,-17.53,82.65
BANKS,6.50,-9.06,66.44
PAINTS,5.67,-15.51,32.73
ELECTRICAL,5.30,-10.18,48.83
INSURANCE,3.91,-0.32,37.82
AUTO,2.79,-45.73,106.55


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3286097.0,22
XR,1345703.0,14
AR,1172571.0,9
X40,800865.0,12
X40N,614648.0,8
OX40N,570797.0,10
XY25,442216.0,8
SR,265402.0,2
X5K,107562.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3404513.0,25
M-SC,1203457.0,15
X-MC,1115063.0,13
X-LC,1058416.0,13
X-SC,528077.0,6
H-MC,344590.0,3
H-LC,283826.0,3
M-LC,281296.0,4
L-SC,251144.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1172187.0      6
           AR         837722.0      5
           XR         804796.0      7
M-SC       XY24       681929.0      6
X-MC       XY24       587816.0      4
X-LC       X40        474603.0      5
           XY24       306289.0      2
X-SC       X40N       287241.0      4
H-SC       OX40N      267384.0      4
           SR         265402.0      2
X-MC       X40        249240.0      6
X-LC       X40N       211348.0      3
H-LC       AR         208013.0      2
H-MC       XY24       180113.0      1
M-MC       XY24       169272.0      1
X-SC       XY24       163814.0      1
X-MC       XY25       161948.0      2
L-SC       XR         161907.0      2
M-SC       OX40N      141822.0      4
           XY25       136311.0      1
           AR         126836.0      2
           XR         116559.0      2
X-MC       X40N       116059.0      1
M-LC       XR         110867.0      1
           X5K        107562.0      1
H-MC       XR          92123.0      1
L-SC       OX40N       89237.0      1
X-SC       X40         77022.0      1
H-LC       X200        75813.0      1
H-MC       OX40N       72354.0      1
X-LC       XY25        66176.0      3
L-MC       XR          59451.0      1
H-SC       MH          57022.0      1
L-LC       XY25        39591.0      1
M-LC       XY25        38190.0      1
           XY24        24677.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
